In [4]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
import matplotlib.pyplot as plt
import numpy as np

In [6]:
vol = np.array([6.26,5.26,5.16,16.2,12.3,23.8,7.30,6.30,6.20])
mu = np.array([0.350,0.250,0.240,4.09,2.50,6.02,1.08,0.980,0.970])
v = np.array([0.96,0.15,0.97,0.39,0.28,0.08,0.14,0.42,0.91])

CorMat = np.array([[1,	0.70,	0.70,	0.17,	0.06,	0.23,	0.52,	0.52,	0.52],
                        [0.70,	1,	0.70,	0.17,	0.06,	0.23,	0.52,	0.52,	0.52],
                        [0.70,	0.70,	1,	0.17,	0.06,	0.23,	0.52,	0.52,	0.52],
                        [0.17,	0.17,	0.17,	1,	0.29,	0.57,	0.50,	0.50,	0.50],
                        [0.06,	0.06,	0.06,	0.29,	1,	0.20,	0.25,	0.25,	0.25],
                        [0.23,	0.23,	0.23,	0.57,	0.20,	1,	0.44,	0.44,	0.44],
                        [0.52,	0.52,	0.52,	0.50,	0.25,	0.44,	1,	0.70,	0.70],
                        [0.52,	0.52,	0.52,	0.50,	0.25,	0.44,	0.70,	1,	0.70],
                        [0.52,	0.52,	0.52,	0.50,	0.25,	0.44,	0.70,	0.70,	1]])

sigma = Cov = np.dot(CorMat,(np.dot(vol,vol.transpose()))) # covariance matrix

q = 0.5                   # set risk factor
num_assets = len(vol)
budget = num_assets // 2  # set budget
penalty = num_assets      # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

In [12]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [10]:
ee = NumPyMinimumEigensolver(qubitOp)
result = ee.run()
print('Obj:', result.eigenvalue + offset)

eigenvalue = ee.compute_minimum_eigenvalue()
print(eigenvalue.eigenvalue)

Obj: (143.99999999999545+0j)
(-7067.545353000008+0j)


In [11]:
print_result(result)

Optimal: selection [0 0 0 0 0 0 0 0 0], value 144.0000

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 0 0 0 0 0 0 0]	144.0000		1.0000
[0 1 1 1 1 1 1 1 1]	18770.3153		0.0000
[1 0 0 0 0 1 0 1 0]	3243.2905		0.0000
[0 1 0 0 0 1 0 1 0]	3243.3905		0.0000
[1 1 0 0 0 1 0 1 0]	5583.9285		0.0000
[0 0 1 0 0 1 0 1 0]	3243.4005		0.0000
[1 0 1 0 0 1 0 1 0]	5583.9385		0.0000
[0 1 1 0 0 1 0 1 0]	5584.0385		0.0000
[1 1 1 0 0 1 0 1 0]	8786.1261		0.0000
[0 0 0 1 0 1 0 1 0]	3625.1731		0.0000
[1 0 0 1 0 1 0 1 0]	5327.0236		0.0000
[0 1 0 1 0 1 0 1 0]	5327.1236		0.0000
[1 1 0 1 0 1 0 1 0]	7890.5237		0.0000
[0 0 1 1 0 1 0 1 0]	5327.1336		0.0000
[1 0 1 1 0 1 0 1 0]	7890.5337		0.0000
[0 1 1 1 0 1 0 1 0]	7890.6337		0.0000
[1 1 1 1 0 1 0 1 0]	11315.5834		0.0000
[0 0 0 0 1 1 0 1 0]	2879.6192		0.0000
[1 0 0 0 1 1 0 1 0]	4448.9120		0.0000
[0 1 0 0 1 1 0 1 0]	4449.0120		0.0000
[1 1 0 0 1 1 0 1 0]	6879.8543		0.0000
[0 0 1 0 1 1

In [14]:
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA


backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

Optimal: selection [0. 0. 0. 0. 0. 0. 0. 0. 0.], value 144.0000

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 0 0 0 0 0 0 0]	144.0000		0.9881
[0 0 0 1 0 0 0 0 0]	679.4454		0.0035
[0 0 0 0 0 0 0 1 0]	682.5554		0.0013
[0 0 1 0 0 0 0 0 0]	683.2954		0.0011
[0 0 0 0 0 0 1 0 0]	682.4554		0.0010
[1 0 0 0 1 0 0 0 0]	1310.5250		0.0007
[0 1 0 0 0 0 0 0 0]	683.2854		0.0006
[0 0 1 0 0 1 0 0 0]	1511.9771		0.0005
[0 0 1 0 1 0 0 0 0]	1310.6350		0.0003
[0 0 0 0 1 0 0 1 0]	1538.8585		0.0003
[1 0 0 0 0 0 0 0 0]	683.1854		0.0003
[1 0 0 0 0 1 0 0 0]	1511.8671		0.0002
[0 0 0 0 1 0 0 0 1]	1538.8685		0.0001
[0 0 0 0 0 1 0 0 0]	677.5154		0.0001
[0 1 0 0 1 0 0 0 0]	1310.6250		0.0001
[0 0 0 0 1 1 0 0 0]	1473.5650		0.0001
[0 1 0 0 0 1 0 0 0]	1511.9671		0.0001
[0 0 0 1 0 0 0 0 1]	1838.5462		0.0001
[1 0 1 0 0 0 0 0 0]	2084.0304		0.0001
[0 0 1 0 0 1 0 0 1]	3243.4105		0.0001
[1 0 1 0 0 0 0 0 1]	3911.8694		0.0001
[1 0 0 0 0 0